In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
import random
import pdb
import torch
from torch.optim import Adam
import gym
import time
import wandb
from spinup import models
core = models

In [7]:
from spinup.utils.mpi_pytorch import setup_pytorch_for_mpi, sync_params, mpi_avg_grads
from spinup.utils.mpi_tools import mpi_fork, mpi_avg, proc_id, mpi_statistics_scalar, num_procs
from spinup.algos.ppo.ppo import ppo
from spinup.algos.sac.sac import sac

In [8]:
class Config(object):
    def __init__(self):
        return None
    def toDict(self):
        pr = {}
        for name in dir(self):
            value = getattr(self, name)
            if not name.startswith('__') and not callable(value) and not name.endswith('_'):
                pr[name] = value
        return pr
    
class TabularLogger(object):
    """
    A text interface logger, outputs mean and std several times per epoch
    """
    def __init__(self):
        self.buffer = {}
        
    def log(dic, commit=False):
        if commit:
            print
        
class Logger(object):
    """
    A logger wrapper for visualized loggers, such as tb or wandb
    Automatically counts steps, epoch, etc. and sets logging interval
    to prevent the log becoming too big
    uses kwargs instead of dict for convenience
    all None valued keys are counters
    """
    def __init__(self, logger):
        self.logger = logger
        self.counters = {'epoch':0}
        self.frequency = 10 # logs per epoch
        
    def log(self, data=None, **kwargs):
        if data is None:
            data = {}
        data.update(kwargs)
        # counting
        for key in data:
            if not key in self.counters:
                self.counters[key] = 0
            self.counters[key] += 1
            
        to_store = {}
        epoch = self.counters['epoch']
        for key in data:
            count = self.counters[key]
            period = count//(epoch+1) + 1
            flag = random.random()< self.frequency/period
            if flag:
                if data[key] is None:
                    to_store[key] = self.counters[key]
                else:
                    to_store[key] = data[key]
                
        if len(to_store) > 0:
            self.logger.log(to_store, commit=True)
        
    def flush(self):
        self.logger.log(data={'epoch':self.counters['epoch']}, commit=True)
        
args = Config()
#args.env="BipedalWalker-v3" # continous action
args.env="Breakout-ram-v0" # discrete action
args.algorithm="sac"
args.name=f"{args.env}_{args.algorithm}"
args.gpu=0
args.seed=0
args.cpu=4
args.steps_per_epoch=5000
args.epochs=500

model_args=Config()
model_args.hidden_sizes=[256]*4
model_args.activation=torch.nn.ReLU
model_args = Config()
model_args.gamma=0.99
model_args.polyak=0.995
model_args.lr=3e-5
model_args.alpha=0.2
model_args.eps=1e-2
args.model_args = model_args.toDict()

# Run
Notice that for breakout, the agent need to press the "fire" button or it gets stuck forever.

## SAC

In [ ]:
run=wandb.init(
    project="RL",
    config=args,
    name=args.name,
    group=args.env,
)
logger = Logger(run)
env = gym.make(args.env)
model = core.MLPDQActorCritic(env.observation_space, env.action_space, logger=logger, **(model_args.toDict()))
result =sac(lambda : gym.make(args.env), model=model, logger=logger, 
           steps_per_epoch=args.steps_per_epoch, epochs=args.epochs)
run.finish()

wandb: ERROR Control-C detected -- Run data was not synced


wandb: wandb version 0.10.29 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



Number of parameters: 	 pi: 99844, 	 q1: 99844, 	 q2: 99844



## PPO

In [ ]:
#mpi_fork(args.cpu)  # run parallel code with mpi
run=wandb.init(
    project="RL",
    config=args,
    name=args.name,
    group=args.env,
)
logger = Logger(run)
result = ppo(lambda : gym.make(args.env), actor_critic=core.MLPVActorCritic,
    ac_kwargs=dict(hidden_sizes=(args.hid,)*args.l, logger=logger,  gamma=args.gamma, 
        seed=args.seed, steps_per_epoch=args.steps_per_epoch, epochs=args.epochs)
run.finish()

# Visualization

In [ ]:
import gym
from IPython import display
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

env = gym.make('Breakout-ram-v0')
state = env.reset()
img = plt.imshow(env.render(mode='rgb_array')) # only call this once
total = 0
for _ in range(2000):
    img.set_data(env.render(mode='rgb_array')) # just update the data
    display.display(plt.gcf())
    display.clear_output(wait=True)
    tmp = torch.tensor(state).float()
    action = model.act(tmp, deterministic=True)
   # action = env.action_space.sample()
    state, reward, done, info  = env.step(action)
    total += reward
    if done:
        print(f"episode len {_}, reward {total}")
        break

In [ ]:
x = env.action_space

In [ ]:
env = gym.make('Breakout-ram-v0')

In [ ]:
env.observation_space.shape

In [ ]:
env.action_space.shape

In [ ]:
env.reset()

In [ ]:
from torch.distributions.categorical import Categorical

In [ ]:
x=Categorical(torch.tensor([1, 0]))
x

In [ ]:
x

In [ ]:
x=torch.randn(3)

In [ ]:
x.argmax()

In [ ]:
torch.multinomial()

In [ ]:

x=Categorical(torch.tensor([0.5, 0.5]))

In [ ]:
y

In [ ]:
x.sample()